In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from datetime import datetime

In [2]:
df_scores = pd.read_csv("compas-scores.csv")
df_scores = df_scores[df_scores.c_charge_degree != 'O']

In [16]:
battery = ['batt']
assault = ['assault', 'abuse']
drugs = ['subst', 'cannabi', 'cocaine', 'deliver', 'del ', 'alprazolam', 'pyrro', 'heroin', 
              'hydro','ethylone', 'drug', 'anabolic', 'steroid',
             'tamine', 'meth', 'codeine', 'lsd',
             'buprenorphine', 'lorazepam','carisoprodol', 'benzyl',
             'clonazepam', 'diazepam', 'morph', 'fluor', ',4', 'fentanyl',
             'phentermine', 'butylone']
driving = ['dui', 'driv', 'veh', 'operat', ' dl', 'd.u.i', 'dwi']
prop = ['burg', 'arson', 'theft', 'property', 'mischief', 'trespass', 'fraud', 'stole']
weapon = ['firearm', 'weapon', 'arm', 'wep']
sexual = ['sex', 'prostitu', 'lewd', 'voyeur', 'porn']
obstruct = ['flee', 'forg', 'tamper', 'false', 'obstruct', 'obey', 'resist', ' id']
domestic = ['dom', 'child']
agg = ['agg']
violent = ['rape', 'assault', 'abuse', 'kidnap', 'murder', 'manslaughter', 'batt', 'assault', 'abuse']
public_order = ['weapon', 'firearm', 'arm', 'dui', 'driv', 'd.u.i.', 'dwi']

In [18]:
print Counter(df_scores['c_charge_desc']).most_common()

[('arrest case no charge', 1857), ('Battery', 1811), ('Possession of Cocaine', 703), ('Grand Theft in the 3rd Degree', 688), ('Driving While License Revoked', 255), ('Driving Under The Influence', 217), ('Felony Battery (Dom Strang)', 174), ('Felony Driving While Lic Suspd', 159), ('Grand Theft (Motor Vehicle)', 139), ('Burglary Unoccupied Dwelling', 132), ('Pos Cannabis W/Intent Sel/Del', 131), ('Possess Cannabis/20 Grams Or Less', 127), ('Aggravated Assault W/Dead Weap', 124), ('Burglary Conveyance Unoccup', 124), ('DUI Property Damage/Injury', 111), ('Possession of Cannabis', 109), ('Aggrav Battery w/Deadly Weapon', 101), ('Felony Petit Theft', 98), ('DUI Level 0.15 Or Minor In Veh', 93), ('Driving License Suspended', 92), ('Uttering a Forged Instrument', 91), ('Resist Officer w/Violence', 90), ('Battery on Law Enforc Officer', 83), ('Poss3,4 Methylenedioxymethcath', 81), ('Possession Of Alprazolam', 79), ('Aggravated Battery / Pregnant', 75), ('Felony Battery w/Prior Convict', 73),

In [17]:
crimes = list(df_scores['c_charge_desc'].unique())
for c in crimes:
    crime = str(c).lower()
    if any(d in crime for d in violent):
        print crime

aggravated assault w/firearm
felony battery w/prior convict
battery
battery on law enforc officer
lewd/lasc battery pers 12+/<16
burglary structure assault/batt
agg battery grt/bod/harm
aggravated battery / pregnant
burglary dwelling assault/batt
felony battery (dom strang)
aggravated assault w/dead weap
aggrav battery w/deadly weapon
assault
burglary conveyance assault/bat
aggravated assault w/dead weap
child abuse
aggravated battery (firearm/actual possession)
burglary with assault/battery
sexual battery / vict 12 yrs +
battery on parking enfor speci
felony battery
aggravated battery
aggrav child abuse-agg battery
aggravated assault
manslaughter w/weapon/firearm
battery on a person over 65
sex battery deft 18+/vict 11-
agg assault w/int com fel dome
battery emergency care provide
aggravated assault w/o firearm
neglect/abuse elderly person
solic to commit battery
aggrav child abuse-causes harm
felony batt(great bodily harm)
agg assault law enforc officer
aggr child abuse-torture,punis

In [9]:
list(df_scores['r_charge_desc'])[1]

'Felony Battery (Dom Strang)'

In [13]:
sex_recid = 0
for i in range(df_scores.shape[0]):
    charge = str(list(df_scores['r_charge_desc'])[i]).lower()
    if any(d in charge for d in sexual):
        print charge, "-----", str(list(df_scores['c_charge_desc'])[i]).lower()

prostitution/lewd act assignation ----- possession of cocaine
sexual battery victim 12 yrs + ----- arrest case no charge
lewd/assignation/prostitution ----- battery
lewd/lasc battery pers 12+/<16 ----- arrest case no charge
loiter solicit act prostitute ----- possession of cocaine
solicit prostitutionviolation ----- possession of cocaine
procure for prostitution ----- possession of cocaine
felony committing prostitution ----- battery
prostitution/lewd act assignation ----- possession of cocaine
sex batt faml/cust vict 12-17y ----- lewd act presence child 16-
prostitution ----- possession of cocaine
prostitution/lewd act assignation ----- battery
transport prostitution ----- driving while license revoked
prostitution/lewd act assignation ----- possession of cocaine
sex offender fail comply w/law ----- possession of cocaine
sexual battery / vict 12 yrs + ----- felony battery (dom strang)
renting for prostitution ----- disorderly conduct
sex battery deft 18+/vict 11- ----- aggrav battery 

In [4]:
#charges = df_scores['c_charge_desc'].unique()
charges = Counter(df_scores['c_charge_desc']).most_common()
sexuals = []
for charge in charges:
    charge = str(charge)
    if any(d in charge.lower() for d in sexual):
        sexuals.append(charge)

print sexuals

["('Prostitution/Lewd Act Assignation', 14)", "('Lewd or Lascivious Molestation', 12)", "('Felony Committing Prostitution', 6)", "('Lewd/Lasc Battery Pers 12+/<16', 5)", "('Possession Child Pornography', 5)", "('Live on Earnings of Prostitute', 5)", "('Sexual Battery / Vict 12 Yrs +', 4)", "('Sexual Performance by a Child', 4)", "('Soliciting For Prostitution', 2)", "('Prostitution/Lewdness/Assign', 2)", "('Offer Agree Secure/Lewd Act', 2)", "('Sex Battery Deft 18+/Vict 11-', 2)", "('Lewdness Violation', 2)", "('Fail to Reg as Sexual Offender', 2)", "('Sex Offender Fail Comply W/Law', 2)", "('Voyeurism', 1)", "('Offer Agree Secure For Lewd Act', 1)", "('Sex Batt Faml/Cust Vict 12-17Y', 1)", "('Procure For Prostitution', 1)", "('Solicit ProstitutionViolation', 1)", "('Aide/Abet Prostitution Lewdness', 1)", "('Lewd Act Presence Child 16-', 1)", "('Lewd/Lasciv Molest Elder Persn', 1)", "('Video Voyeur-<24Y on Child >16', 1)", "('Unlawful Sexual Activity', 1)", "('Fail Sex Offend Report By

In [5]:
df_sexual = pd.read_csv('compas-r-sexual.csv')
sex_recid = 0
sex_double = 0
for i in range(len(df_sexual['is_sexual_recid'])):
    if int(df_sexual['is_sexual_recid'][i]) == 1:
        sex_recid += 1
        if int(df_sexual['prior_sexual'][i]) == 1:
            sex_double += 1
        #print df_scores[df_scores.id == df_sexual['id'][i]]
print sex_recid, sex_double

692 5
